In [2]:
## Classification of MNIST dataset 
## with the convolutional neural network know as LeNet5.
## This script also combines various
## packages from the Julia ecosystem  with Flux.
using Flux
using Flux.Data: DataLoader
using Flux.Optimise: Optimiser, WeightDecay
using Flux: onehotbatch, onecold, logitcrossentropy
using Statistics, Random
using Parameters: @with_kw
using Logging: with_logger, global_logger
using TensorBoardLogger: TBLogger, tb_overwrite, set_step!, set_step_increment!
import ProgressMeter
import MLDatasets
import DrWatson: savename, struct2dict
import BSON
using CUDAapi

In [3]:
# LeNet5 "constructor". 
# The model can be adapted to any image size
# and number of output classes.
function LeNet5(; imgsize=(64,64,3), nclasses=10) 
    out_conv_size = (imgsize[1]÷4 - 3, imgsize[2]÷4 - 3, 16)
    
    return Chain(
            x -> reshape(x, imgsize..., :),
            Conv((5, 5), imgsize[end]=>6, relu),
            MaxPool((2, 2)),
            Conv((5, 5), 6=>16, relu),
            MaxPool((2, 2)),
            x -> reshape(x, :, size(x, 4)),
            Dense(prod(out_conv_size), 120, relu), 
            Dense(120, 84, relu), 
            Dense(84, nclasses)
          )
end

LeNet5 (generic function with 1 method)

In [4]:
function get_data(args)
    xtrain, ytrain = MLDatasets.CIFAR10.traindata(Float32, dir=args.datapath)
    xtest, ytest = MLDatasets.CIFAR10.testdata(Float32, dir=args.datapath)

    xtrain = reshape(xtrain, 64, 64, 3, :)
    xtest = reshape(xtest, 64, 64, 3, :)

    ytrain, ytest = onehotbatch(ytrain, 0:9), onehotbatch(ytest, 0:9)

    train_loader = DataLoader(xtrain, ytrain, batchsize=args.batchsize, shuffle=true)
    test_loader = DataLoader(xtest, ytest,  batchsize=args.batchsize)
    
    return train_loader, test_loader
end

get_data (generic function with 1 method)

In [5]:
loss(ŷ, y) = logitcrossentropy(ŷ, y)

loss (generic function with 1 method)

In [6]:
function eval_loss_accuracy(loader, model, device)
    l = 0f0
    acc = 0
    ntot = 0
    for (x, y) in loader
        x, y = x |> device, y |> device
        ŷ = model(x)
        l += loss(ŷ, y) * size(x)[end]        
        acc += sum(onecold(ŷ |> cpu) .== onecold(y |> cpu))
        ntot += size(x)[end]
    end
    return (loss = l/ntot |> round4, acc = acc/ntot*100 |> round4)
end

eval_loss_accuracy (generic function with 1 method)

In [7]:
## utility functions

num_params(model) = sum(length, Flux.params(model)) 

num_params (generic function with 1 method)

In [8]:
round4(x) = round(x, digits=4)

round4 (generic function with 1 method)

In [9]:
# arguments for the `train` function 
@with_kw mutable struct Args
    η = 3e-4             # learning rate
    λ = 0                # L2 regularizer param, implemented as weight decay
    batchsize = 128      # batch size
    epochs = 20          # number of epochs
    seed = 0             # set seed > 0 for reproducibility
    cuda = true          # if true use cuda (if available)
    infotime = 1 	     # report every `infotime` epochs
    checktime = 10        # Save the model every `checktime` epochs. Set to 0 for no checkpoints.
    tblogger = false      # log training with tensorboard
    savepath = nothing    # results path. If nothing, construct a default path from Args. If existing, may overwrite
    datapath = joinpath(homedir(), "Datasets", "CIFAR10") # data path: change to your data directory 
end

Args

In [10]:
function train(; kws...)
    args = Args(; kws...)
    args.seed > 0 && Random.seed!(args.seed)
    use_cuda = args.cuda && CUDAapi.has_cuda_gpu()
    if use_cuda
        device = gpu
        @info "Training on GPU"
    else
        device = cpu
        @info "Training on CPU"
    end

    ## DATA
    train_loader, test_loader = get_data(args)
    @info "Dataset MNIST: $(train_loader.nobs) train and $(test_loader.nobs) test examples"

    ## MODEL AND OPTIMIZER
    model = LeNet5() |> device
    @info "LeNet5 model: $(num_params(model)) trainable params"    
    
    ps = Flux.params(model)  

    opt = ADAMW(args.η) 
    if args.λ > 0 
        opt = Optimiser(opt, WeightDecay(args.λ))
    end
    
    ## LOGGING UTILITIES
    if args.savepath == nothing
        experiment_folder = savename("lenet", args, scientific=4,
                    accesses=[:batchsize, :η, :seed, :λ]) # construct path from these fields
        args.savepath = joinpath("runs", experiment_folder)
    end
    if args.tblogger 
        tblogger = TBLogger(args.savepath, tb_overwrite)
        set_step_increment!(tblogger, 0) # 0 auto increment since we manually set_step!
        @info "TensorBoard logging at \"$(args.savepath)\""
    end
    
    function report(epoch)
        train = eval_loss_accuracy(train_loader, model, device)
        test = eval_loss_accuracy(test_loader, model, device)        
        println("Epoch: $epoch   Train: $(train)   Test: $(test)")
        if args.tblogger
            set_step!(tblogger, epoch)
            with_logger(tblogger) do
                @info "train" loss=train.loss  acc=train.acc
                @info "test"  loss=test.loss   acc=test.acc
            end
        end
    end
    
    ## TRAINING
    @info "Start Training"
    report(0)
    for epoch in 1:args.epochs
        p = ProgressMeter.Progress(length(train_loader))

        for (x, y) in train_loader
            x, y = x |> device, y |> device
            gs = Flux.gradient(ps) do
                ŷ = model(x)
                loss(ŷ, y)
            end
            Flux.Optimise.update!(opt, ps, gs)
            ProgressMeter.next!(p)   # comment out for no progress bar
        end
        
        epoch % args.infotime == 0 && report(epoch)
        if args.checktime > 0 && epoch % args.checktime == 0
            !ispath(args.savepath) && mkpath(args.savepath)
            modelpath = joinpath(args.savepath, "model.bson") 
            let model=cpu(model), args=struct2dict(args)
                BSON.@save modelpath model epoch args
            end
            @info "Model saved in \"$(modelpath)\""
        end
    end
end

train (generic function with 1 method)

In [11]:
train()

┌ Info: Training on GPU
└ @ Main In[10]:7
┌ Warning: The file "/home/subhaditya/Datasets/CIFAR10/cifar-10-batches-bin/data_batch_1.bin" does not exist, even though the dataset-specific folder does. This is an unusual situation that may have been caused by a manual creation of an empty folder, or manual deletion of the given file "cifar-10-batches-bin/data_batch_1.bin".
└ @ MLDatasets /home/subhaditya/.julia/packages/MLDatasets/GU5Hj/src/download.jl:34
┌ Info: Retriggering DataDeps.jl for "CIFAR10" to "/home/subhaditya/Datasets/CIFAR10".
└ @ MLDatasets /home/subhaditya/.julia/packages/MLDatasets/GU5Hj/src/download.jl:39


This program has requested access to the data dependency CIFAR10.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: The CIFAR-10 dataset
Authors: Alex Krizhevsky, Vinod Nair, Geoffrey Hinton
Website: https://www.cs.toronto.edu/~kriz/cifar.html
Reference: https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf

[Krizhevsky, 2009]
    Alex Krizhevsky.
    "Learning Multiple Layers of Features from Tiny Images",
    Tech Report, 2009.

The CIFAR-10 dataset is a labeled subsets of the 80
million tiny images dataset. It consists of 60000
32x32 colour images in 10 classes, with 6000 images
per class.

The compressed archive file that contains the
complete dataset is available for download at the
offical website linked above; specifically the binary
version for C programs. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of CIFAR-10 aren't really
explicit about an

stdin>  y


┌ Info: Downloading
│   source = https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz
│   dest = /home/subhaditya/Datasets/CIFAR10/cifar-10-binary.tar.gz
│   progress = 0.0066
│   time_taken = 5.42 s
│   time_remaining = 809.12 s
│   average_speed = 203.878 KiB/s
│   downloaded = 1.078 MiB
│   remaining = 161.096 MiB
│   total = 162.174 MiB
└ @ HTTP /home/subhaditya/.julia/packages/HTTP/GkPBm/src/download.jl:119
┌ Info: Downloading
│   source = https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz
│   dest = /home/subhaditya/Datasets/CIFAR10/cifar-10-binary.tar.gz
│   progress = 0.0156
│   time_taken = 10.54 s
│   time_remaining = 664.49 s
│   average_speed = 246.014 KiB/s
│   downloaded = 2.531 MiB
│   remaining = 159.643 MiB
│   total = 162.174 MiB
└ @ HTTP /home/subhaditya/.julia/packages/HTTP/GkPBm/src/download.jl:119
┌ Info: Downloading
│   source = https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz
│   dest = /home/subhaditya/Datasets/CIFAR10/cifar-10-binary.tar.gz
│   

InterruptException: InterruptException: